In [12]:
import csv
import string
from sklearn.model_selection import train_test_split

In [11]:


def process_mails(train_data):
  ham_dict = {}
  spam_dict = {}
  ham_count=0
  spam_count=0
    
  for row in train_data:
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in row[1]:
      if char not in punctuations:
        no_punct = no_punct + char
    no_punct = no_punct.lower()
    text_ham_dict = {}
    text_spam_dict = {}

    if row[0]=="ham":
      ham_count+=1 
      for word in no_punct.split():
        if word in ham_dict.keys() and word not in text_ham_dict.keys():
          ham_dict[word] = ham_dict[word] + 1
        elif word not in text_ham_dict.keys():
          ham_dict[word] = 1
        text_ham_dict[word] = 1
    else:
      spam_count+=1
      for word in no_punct.split():
        if word in spam_dict.keys() and word not in text_spam_dict.keys():
          spam_dict[word] = spam_dict[word] + 1
        elif word not in text_spam_dict.keys():
          spam_dict[word] = 1
        text_spam_dict[word] = 1
  return ham_dict, spam_dict, ham_count, spam_count




def data_loader(file):
  with open(file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    data = []
    for row in csv_reader:
      data.append([row[0],row[1]])
    # print(data)
    train_data, test_data = train_test_split(data, test_size=0.3)
  return train_data, test_data




In [13]:
train_data, test_data = data_loader('Data_p1.csv')
ham_dict, spam_dict, ham_count, spam_count = process_mails(train_data)


In [14]:
ham_dict

{'yeah': 58,
 'we': 171,
 'can': 245,
 'probably': 24,
 'swing': 6,
 'by': 87,
 'once': 27,
 'my': 403,
 'roommate': 2,
 'finishes': 3,
 'up': 194,
 'with': 166,
 'his': 43,
 'girl': 15,
 'princess': 23,
 'i': 1115,
 'like': 144,
 'to': 857,
 'make': 58,
 'love': 117,
 'ltgt': 139,
 'times': 17,
 'per': 9,
 'night': 60,
 'hope': 72,
 'thats': 57,
 'not': 276,
 'a': 603,
 'problem': 20,
 'o': 10,
 'ic': 2,
 'lol': 53,
 'should': 59,
 'play': 14,
 '9': 22,
 'doors': 2,
 'sometime': 3,
 'yo': 24,
 'how': 176,
 'tall': 1,
 'are': 264,
 'you': 913,
 'uncle': 9,
 'boye': 3,
 'need': 114,
 'movies': 6,
 'oh': 82,
 'guide': 4,
 'me': 475,
 'plus': 12,
 'know': 167,
 'torrents': 1,
 'particularly': 1,
 'legal': 2,
 'here': 78,
 'and': 471,
 'the': 578,
 'system': 5,
 'is': 419,
 'slowing': 1,
 'down': 37,
 'what': 172,
 'do': 225,
 'have': 272,
 'gr8': 8,
 'day': 126,
 'started': 13,
 'cos': 59,
 'dont': 164,
 'meet': 48,
 'online': 15,
 'was': 135,
 'honey': 5,
 'moon': 3,
 'ok': 195,
 'guessi

In [10]:
def classify(text):
  word_dict = {}
  p_text_spam = 1
  p_text_ham = 1
  p_text = 1
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  no_punct = ""
  for char in text:
    if char not in punctuations:
      no_punct = no_punct + char
  no_punct = no_punct.lower()
  for word in no_punct.split():
    if word not in word_dict.keys():
      word_dict[word] = 1
      spam_dict_word = 1
      ham_dict_word = 1
      if word in spam_dict.keys():
        spam_dict_word = spam_dict[word] + 1
      if word in ham_dict.keys():
        ham_dict_word = ham_dict[word] + 1
      p_text_spam *= (spam_dict_word)/(spam_count)
      p_text_ham *= (ham_dict_word)/(ham_count)
      p_text *= (ham_dict_word+spam_dict_word)/(ham_count+spam_count)
  for word in spam_dict.keys():
    if word not in word_dict.keys():
      word_dict[word] = 1
      spam_dict_word = 1
      ham_dict_word = 1
      if word in spam_dict.keys():
        spam_dict_word = spam_dict[word] + 1
      if word in ham_dict.keys():
        ham_dict_word = ham_dict[word] + 1
      p_text_spam *= 1-(spam_dict_word)/(spam_count+1)
      p_text_ham *= 1-(ham_dict_word)/(ham_count+1)
      p_text *= 1-(ham_dict_word+spam_dict_word)/(ham_count+spam_count+2)
  for word in ham_dict.keys():
    if word not in word_dict.keys():
      word_dict[word] = 1
      spam_dict_word = 1
      ham_dict_word = 1
      if word in spam_dict.keys():
        spam_dict_word = spam_dict[word] + 1
      if word in ham_dict.keys():
        ham_dict_word = ham_dict[word] + 1
      p_text_spam *= 1-(spam_dict_word)/(spam_count+1)
      p_text_ham *= 1-(ham_dict_word)/(ham_count+1)
      p_text *= 1-(ham_dict_word+spam_dict_word)/(ham_count+spam_count+2)

  p_spam = spam_count/(spam_count+ham_count)
  p_ham = ham_count/(spam_count+ham_count)
  if p_text==0:
    print(no_punct)
    return 0
  p_spam_text = p_text_spam*p_spam/p_text
  p_ham_text = p_text_ham*p_ham/p_text
  p_spam_text = p_spam_text/(p_spam_text+p_ham_text)
  p_ham_text = 1 - p_spam_text
  
  if p_spam_text > 0.5:
    return 1
  else:
    return 0

def test(test_data):
  hh = 0
  ss = 0
  sh = 0
  hs = 0

  for row in test_data:
    res = classify(row[1])

    if row[0]=="ham" and res == 0:
      hh += 1
    elif row[0]=="ham" and res == 1:
      sh += 1
    elif row[0]=="spam" and res == 1:
      ss += 1
    else:
      hs += 1

  acc = (hh+ss)/(hh+ss+sh+hs)
  print("Accuracy = ", acc)
  print("Confusion Matrix \n -----------------------------------")
  print("\tPrediction-SPAM    Prediction-HAM")
  print(" -----------------------------------")
  print("Actual-SPAM\t",ss,"\t",hs)
  print("Actual-HAM\t",sh,"\t",hh)
  print(" -----------------------------------")

  

In [6]:
a = classify("A Boy loved a gal. He propsd bt she didnt mind. He gv lv lttrs, Bt her frnds threw thm. Again d boy decided 2 aproach d gal , dt time a truck was speeding towards d gal. Wn it was about 2 hit d girl,d boy ran like hell n saved her. She asked 'hw cn u run so fast?' D boy replied \Boost is d secret of my energy\" n instantly d girl shouted \"our energy\" n Thy lived happily 2gthr drinking boost evrydy Moral of d story:- I hv free msgs:D;): gud ni8")
a

0

In [7]:
b = classify("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's")
b

1

In [8]:
test(test_data)

Accuracy =  0.9730861244019139
Confusion Matrix 
 -----------------------------------
	Prediction-SPAM    Prediction-HAM
 -----------------------------------
Actual-SPAM	 195 	 39
Actual-HAM	 6 	 1432
 -----------------------------------
